In [14]:
from pydantic_ai import Agent
from pydantic import BaseModel, Field
from typing import List, Optional

In [30]:
class answer(BaseModel):
    answer: str = Field(description="The answer to the question")
    source: Optional[str] = Field(description="The source of the answer, if applicable")

class context(BaseModel):
    country: str = Field(description="The country to answer the question about")


In [47]:
model_name = "gemini-flash-latest"

agent = Agent(
    model=model_name,
    output_type=answer,
    system_prompt="You are a helpful assistant. Use tools when needed.",
    model_settings={
        "temperature": 0.5,
        "top_p": 0.95,
        "top_k": 40,
        "max_tokens": 1000,
        "stop_sequences": ["\n\n"],
        "response_mime_type": "application/json",
    },
    deps_type=context,
)

@agent.system_prompt
def add_country_name(ctx):
    return f"The country is {ctx.deps.country}"

COUNTRY_DATA = {
    "France": {
        "capital": "Paris",
        "source": "https://en.wikipedia.org/wiki/France",
    }
}

@agent.tool
def get_capital(ctx, country: str) -> str:
    return COUNTRY_DATA.get(country, {}).get("capital", "Unknown")

@agent.tool
def get_source(ctx, country: str) -> str:
    return COUNTRY_DATA.get(country, {}).get("source", "N/A")

result = await agent.run(
    "What is the capital? Use tools and include the source.",
    deps=context(country="France"),
)



In [48]:
result.output

answer(answer='The capital of France is Paris.', source='https://en.wikipedia.org/wiki/France')